In [133]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.firefox.options import Options
from selenium.webdriver import Firefox
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC

import urllib.request
from bs4 import BeautifulSoup
import re
import nltk
from collections import Counter
import datetime
from tqdm import tqdm
import time 

url = "https://opportunities.digitalnovascotia.com"
options = Options()
options.add_argument('-headless')
driver = Firefox(executable_path='geckodriver', options=options)
driver.get(url)
print(f'ready: {url}')

ready: https://opportunities.digitalnovascotia.com


In [134]:
count = 1
while True:
    try:
        driver.find_element_by_xpath('//*[@id="jobify_widget_jobs-2"]/div/div/div/div/a').click()
        time.sleep(2)
        count += 1
    except Exception as e:
        print(e)
        print(f'Done loading {count} pages')
        break

# create a list of the most recent jobs
links = []
soup = BeautifulSoup(driver.page_source, 'html.parser')
for a in soup.find_all('a',{'href':re.compile('https://opportunities.digitalnovascotia.com/job/')}):
    links.append(a['href'])
print(f'{len(set(links))} total jobs')

Message: Element <a class="load_more_jobs" href="#"> could not be scrolled into view

Done loading 19 pages
365 total resumes


In [ ]:
holding = []
count = 1
for line in tqdm(links):
    pages = urllib.request.urlopen(line).read()
    souper = BeautifulSoup(pages, 'lxml')

    for script in souper(["script", "style"]):
        script.replace_with(" ")
        script.decompose()

    text = souper.get_text(" ")

    lines = (line.strip() for line in text.splitlines())
    chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
    text = "  ".join(chunk for chunk in chunks if chunk)

    holding.append(text)

words = [s.lower().split() for s in holding if s]

check = [
    'javascript',
    'python',
    'java',
    'golang',
    'php',
    'typescript',
    'r',
    'c#',
    'scala',
    'swift',
    'c++',
    'c',
    'bash',
    'elixir',
    'kotlin',
    'node',
    'powershell',
    'ruby',
    'shell',
    'ansible',
    'css',
    'erlang',
    'glsl',
    'groovy',
    'haskell',
    'html',
    'jsx',
    'objective-c',
    'perl',
    'pug',
    'rust',
    'sass',
    'spark',
    'sql',
    'stan',
    'vba'
]

frequency = {}
for c in check:
    count = 1
    for w in words:
        if c in w:
            frequency[c] = count
            count += 1
frequency_sum = sum([i[1] for i in frequency.items()])
ud = {}
for i in frequency.items():
    ud[i[0]] = round(i[1]/frequency_sum, 2)